In [44]:
import os
os.chdir('F:\\PlaceCom\\Process\\Screening Test - DS')

In [45]:
import pandas as pd
import json
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


In [46]:
# Load the JSON file
with open('a.txt') as file:
    json_str = file.read()


In [47]:
# Parse the JSON string into a Python dictionary
params = json.loads(json_str)
params.keys()

dict_keys(['session_name', 'session_description', 'design_state_data'])

In [48]:
params

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name'

In [55]:
# Step 1: Read the target and type of regression to be run
target = params['design_state_data']['target']['target']
regression_type = params['design_state_data']['target']['type']

# Extract feature handling parameters
feature_handling = params['design_state_data']['feature_handling']

In [56]:
# Step 2: Read the dataset and apply missing imputation
df = pd.read_csv(params['design_state_data']['session_info']['dataset'])
#features = params['design_state_data']['feature_handling']
#missing_imputation = params.get("missing_imputation", "mean")
#imputer = SimpleImputer(strategy=missing_imputation)
#df['feature_handling'] = imputer.fit_transform(df['feature_handling'])
#df['sepal_length'] = imputer.fit_transform(df[['sepal_length']])
#df['sepal_width'] = imputer.fit_transform(df[['sepal_width']])
#df['petal_length'] = imputer.fit_transform(df[['petal_length']])
#df['petal_width'] = imputer.fit_transform(df[['petal_width']])

In [57]:
for col in df.columns:
    # Read feature details
    feature_details = feature_handling[col]['feature_details']
    # Impute missing values
    if 'missing_values' in feature_details and feature_details['missing_values'] == 'Impute':
        if feature_details['impute_with'] == 'Average of values':
            df[col].fillna((df[col].mean()), inplace=True)
        elif feature_details['impute_with'] == 'custom':
            impute_val = feature_details['impute_value']
            df[col].fillna(impute_val, inplace=True)

In [58]:
feature_handling[col]

{'feature_name': 'species',
 'is_selected': True,
 'feature_variable_type': 'text',
 'feature_details': {'text_handling': 'Tokenize and hash', 'hash_columns': 0}}

In [43]:
params['design_state_data']['feature_handling']

{'sepal_length': {'feature_name': 'sepal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'Average of values',
   'impute_value': 0}},
 'sepal_width': {'feature_name': 'sepal_width',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'custom',
   'impute_value': -1}},
 'petal_length': {'feature_name': 'petal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': '

In [40]:
# Loop over each feature in the JSON file
for feature_name, feature in params['design_state_data']['feature_handling'].items():
    # Check if feature is selected
    if feature['is_selected']:
        # Check the missing values imputation method
        missing_value_imputation = feature['feature_details']['missing_values']
        if missing_value_imputation  == 'Impute':
            print("yes impute")
            impute_with = feature['feature_details']['impute_with']
            impute_value = feature['feature_details']['impute_value']
            if impute_with == 'Average of values':
                df[feature_name] = df[feature_name].fillna(df[feature_name].mean())
            elif impute_with == 'custom':
                df[feature_name] = df[feature_name].fillna(impute_value)
        else:
            print("no")

yes impute
yes impute
yes impute
yes impute


KeyError: 'missing_values'

In [32]:
params['design_state_data']['session_info']['dataset']

'iris_modified.csv'

In [33]:
# Step 3: Compute feature reduction
reduction_method = params.get('feature_reduction', 'no_reduction')
if reduction_method == 'corr_with_target':
    corr = data[features + [target]].corr()
    corr_target = corr[target]
    features = features[corr_target.abs().sort_values(ascending=False).index]
elif reduction_method == 'tree_based':
    model = RandomForestRegressor()
    model.fit(data[features], data[target])
    importances = model.feature_importances_
    features = features[importances.argsort()[::-1]]
elif reduction_method == 'pca':
    n_components = params.get('n_components', 2)
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data[features])
    features = ['pca_{}'.format(i) for i in range(n_components)]
    data = pd.DataFrame(data_pca, columns=features)
else:
    pass  # no feature reduction

In [34]:
# Step 4: Build the models
prediction_type = params['design_state_data']['target']["prediction_type"]
models = []
if prediction_type == "Regression":
    if regression_type == "Linear":
        models.append(GridSearchCV(LinearRegression(),
                                   params["linear_regression_params"],
                                   scoring="neg_mean_squared_error"))
    elif regression_type == "Ridge":
        models.append(GridSearchCV(Ridge(),
                                   params["ridge_regression_params"],
                                   scoring="neg_mean_squared_error"))
    elif regression_type == "Lasso":
        models.append(GridSearchCV(Lasso(),
                                   params["lasso_regression_params"],
                                   scoring="neg_mean_squared_error"))
elif prediction_type == "Tree-based":
    if regression_type == "Decision Tree":
        models.append(GridSearchCV(DecisionTreeRegressor(),
                                   params["decision_tree_params"],
                                   scoring="neg_mean_squared_error"))
    elif regression_type == "Random Forest":
        models.append(GridSearchCV(RandomForestRegressor(),
                                   params["random_forest_params"],
                                   scoring="neg_mean_squared_error"))
    elif regression_type == "Gradient Boosting":
        models.append(GridSearchCV(GradientBoostingRegressor(),
                                   params["gradient_boosting_params"],
                                   scoring="neg_mean_squared_error"))

In [35]:
# Step 5: Fit and predict on each model
y_pred = pd.DataFrame({target: df[target]})  # Initialize y_pred with the true target values
for model in models:
    model.fit(df[features], df[target])
    y_pred[model.__class__.__name__] = model.predict(df[features])

In [36]:
# Step 6: Evaluate the model and log the metrics
for model_name in y_pred.columns[1:]:
    y_true = y_pred[target]
    y_model = y_pred[model_name]
    mse = mean_squared_error(y_true, y_model)
    r2 = r2_score(y_true, y_model)
    print(f"{model_name} - MSE: {mse:.4f}, R2: {r2:.4f}")

    # Log the metrics
    with open("metrics.log", "a") as f:
        f.write(f"{model_name} - MSE: {mse:.4f}, R2: {r2:.4f}\n")

In [37]:
y_pred

,petal_width
0,0.2
1,0.2
2,0.2
3,0.2
4,0.2
...,...
145,2.3
146,1.9
147,2.0
148,2.3


In [38]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
